In [2]:
import pandas as pd
from astroquery.mast import Observations
from astropy.coordinates import SkyCoord
import astropy.units as u
import random
from astropy.table import Table
from astropy.io import fits
from astropy.wcs import WCS
from astropy.nddata import Cutout2D
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from joblib import Parallel, delayed

import dask.dataframe as dd

## Importing GZ Manifest

In [4]:
manifest_folder = 'C:/Users/oryan/Documents/esac-project/all-gz-data/'

In [5]:
manifest_df = pd.read_csv(f'{manifest_folder}/all-gz-training-set.csv', index_col = 0)

In [6]:
manifest_df

,zooniverse_id,RA,Dec,interaction
126051,588023046405095627,09:23:29.96,+20:10:09.2,0
531256,587736546311995692,15:08:46.29,+05:24:15.4,0
844518,587742550152184147,15:53:43.26,+11:44:51.2,0
1415,587736584977449086,245.30499,24.155207,1
732911,587731869626466442,10:08:01.48,+50:24:24.7,0
...,...,...,...,...
1882,587739505547608086,195.59517,31.966227000000003,1
3554,587729388219924608,154.92836,57.423485,1
569006,587739844317347919,15:41:00.67,+15:58:01.8,0
884015,587726879949914462,21:16:53.16,-05:53:57.5,0


In [7]:
rand_int = random.randint(0,len(manifest_df))

In [8]:
row = manifest_df.iloc[rand_int]
row

zooniverse_id    587735491357704320
RA                      14:44:17.24
Dec                     +44:55:21.8
interaction                       0
Name: 502608, dtype: object

In [9]:
zooniverse_id = manifest_df.zooniverse_id.iloc[rand_int]
ra = manifest_df.RA.iloc[rand_int]
dec = manifest_df.Dec.iloc[rand_int]

In [10]:
print(f'{zooniverse_id} : ({ra}, {dec})')

587735491357704320 : (14:44:17.24, +44:55:21.8)


In [11]:
if ':' in ra:
    coord = SkyCoord(ra, dec, frame='fk5', unit=(u.hourangle, u.deg))
elif 'm' in ra:
    coord = SkyCoord(ra, dec, frame='fk5',unit = (u.hourangle, u.deg))
else:
    coord = SkyCoord(ra = float(ra) * u.deg, dec = float(dec) * u.deg, frame='fk5')

In [12]:
for _, i in manifest_df.iterrows():
    print(i.RA)
    break

09:23:29.96


In [13]:
def mask_creator(i):
    ra = i.RA
    dec = i.Dec
    
    if ':' in ra:
        coord = SkyCoord(ra, dec, frame='fk5', unit=(u.hourangle, u.deg))
    elif 'm' in ra:
        coord = SkyCoord(ra, dec, frame='fk5',unit = (u.hourangle, u.deg))
    else:
        coord = SkyCoord(ra = float(ra) * u.deg, dec = float(dec) * u.deg, frame='fk5')
        
    obs_table = Observations.query_criteria(
        coordinates = coord,
        radius = 20 * u.arcsec,
        dataproduct_type = 'image',
        obs_collection = 'HST',
        instrument_name='ACS/WFC',
        calib_level = 3,
        filters = ['F814W']
        )
    
    if len(obs_table) > 0:
        return True
    else:
        return False

In [16]:
def mask_creator2(df):
    return df.apply(mask_creator, axis=1)

In [15]:
df_partitioned = dd.from_pandas(manifest_df, npartitions = 8)

In [18]:
p = df_partitioned.map_partitions(mask_creator2, meta = (None, bool))

In [19]:
p.compute()

KeyboardInterrupt: 

In [ ]:
manifest_checked = manifest_df.assign(in_hubble = manifest_df.apply(lambda row: mask_creator(row), axis = 1))

In [ ]:
manifest_checked.in_hubble.value_counts()